In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_fscore_support
)

input_jsonl_path = "//teamspace/studios/this_studio/Inferences/test2.jsonl"  
label_map = {-1.0: 0, 0.0: 1, 1.0: 2}
label_names = ["negative", "neutral", "positive"]

# ---------- LOAD DATA ----------
y_true = []
y_pred = []

with open(input_jsonl_path, 'r') as f:
    for line in f:
        try:
            item = json.loads(line)
            aspect = item["aspect"]
            gt_raw = item["sentiment_dict"].get(aspect, None)
            pred = item["predicted_sentiment"].get(aspect, None)

            if gt_raw is None or pred is None:
                continue

            gt_mapped = label_map.get(gt_raw)
            if gt_mapped is None:
                continue

            y_true.append(gt_mapped)
            y_pred.append(pred)

        except Exception as e:
            print(f"Skipping line: {e}")
            continue

# ---------- METRICS ----------
report = classification_report(y_true, y_pred, target_names=label_names, digits=4, output_dict=True)
conf_matrix = confusion_matrix(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

print("Classification Report (weighted):")
print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
print("Weighted Precision:", round(precision, 4))
print("Weighted Recall:   ", round(recall, 4))
print("Weighted F1 Score: ", round(f1, 4))

# ---------- VISUALIZATION ----------
from matplotlib.colors import LinearSegmentedColormap
pale_yellow_cmap = LinearSegmentedColormap.from_list("pale_yellow_shades", ["#FFF9C4", "#FFEB3B", "#FFEB8C", "#FFCC00"])

#Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=pale_yellow_cmap, 
            xticklabels=label_names, yticklabels=label_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

#Class Distribution 
counts_true = Counter(y_true)
counts_pred = Counter(y_pred)

labels = label_names
true_counts = [counts_true.get(i, 0) for i in range(3)]
pred_counts = [counts_pred.get(i, 0) for i in range(3)]

x = range(len(labels))
width = 0.35

plt.figure(figsize=(7, 5))
plt.bar(x, true_counts, width, label='Ground truth', color='#FFD700')  # Yellow
plt.bar([p + width for p in x], pred_counts, width, label='Predicted', color='#FF8C00')  # Orange
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Class distribution: Ground truth vs Predicted')
plt.xticks([p + width / 2 for p in x], labels)
plt.legend()
plt.tight_layout()
plt.show()